In [1]:
cd /content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification

/content/drive/Othercomputers/내 컴퓨터/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification


# Sentence Classification

* cleaning and basic pre-processing of text
* building a vocabulary, and creating iterators using TorchText
* building a sequnce model - LSTM using Pytorch to predict labels

In [2]:
# 파일 경로 확인
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/01_pytorch_text_classification_torchtext_lstm.ipynb
/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/archive/glove.6B.100d.txt
/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/archive/glove.6B.200d.txt
/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/archive/glove.6B.50d.txt
/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/nlp-getting-started/sample_submission.csv
/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/nlp-getting-started/train.csv
/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/nlp-getting-started/test.csv


In [3]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

import torch
from torchtext import data
import torch.nn as nn

In [4]:
# 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/nlp-getting-started/train.csv')
test = pd.read_csv('/content/drive/MyDrive/kimjuyeon/NLP 스터디/코드 공부/o1_text_classification/nlp-getting-started/test.csv')

# Data Pre-Processing


In [5]:
train.shape

(7613, 5)

In [6]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


* target column이 text의 label을 의미함.
* label == 1 : if the Tweet is about disasters
* label == 0 : if the Tweet is not about disasters

-> 우리는 text와 target column에만 관심이 있기 때문에 나머지는 drop 해줄 것임.


In [7]:
# 'id', 'keyword', 'location' column을 drop
train.drop(columns=['id', 'keyword', 'location'], inplace=True)

In [8]:
train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


* classification 알고리즘이 관련 없는 정보와 혼동되지 않도록 텍스트를 정리하고 수정

In [14]:
# to clean data
def normalise_text(text):
  text = text.str.lower() # lowercase
  text = text.str.replace(r"\#", "") # replace hashtage
  text = text.str.replace(r"http\S+", "URL") # remove URL addresses
  text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
  text = text.str.replace("\s{2,}", " ")
  return text

In [15]:
train["text"] = normalise_text(train["text"])

<ipython-input-14-04169255ccb2>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"\#", "") # replace hashtage
<ipython-input-14-04169255ccb2>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"http\S+", "URL") # remove URL addresses
<ipython-input-14-04169255ccb2>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
<ipython-input-14-04169255ccb2>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace("\s{2,}", " ")


In [16]:
train['text'].head()

0    our deeds are the reason of this earthquake ma...
1                forest fire near la ronge sask canada
2    all residents asked to 'shelter in place' are ...
3    13 000 people receive wildfires evacuation ord...
4    just got sent this photo from ruby alaska as s...
Name: text, dtype: object

* data를 train, valid data로 나누기

In [22]:
train_df, valid_df = train_test_split(train, random_state=0)

In [23]:
train_df.head()

,text,target
5244,refugio oil spill may have been costlier bigge...,1
4860,julian knight scvsupremecourt dismisses mass m...,1
6538,electricity cant stop scofield nigga survived ...,0
5175,meek mill begging nicki minaj to let him oblit...,0
5820,china s stock market crash this summer has spa...,0


In [25]:
valid_df.head()

,text,target
311,katiekatcubs you already know how this shit g...,0
4970,lemairelee danharmon people near meltdown com...,0
527,1 6 tix calgary flames vs col avalanche presea...,0
6362,if you ever think you running out of choices i...,0
800,if you dotish to blight your car go right ahea...,0


* 결과를 재생성하기 위해서 랜덤 시드 고정

In [26]:
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False